# Декартово дерево

Занятия на ДД идут сразу после занятий по ДО и частично на них ссылаются. Прочитайте сначала конспект про ДО.

Рене Декарт — великий французский математик и философ XVII века. Рене Декарт не является создателем декартова дерева, но он является создателем, в частности, декартовой системы координат, которую мы все знаем и любим. Нанесем на неё набор точек. $x$ назовем ключем, а $y$ приоритетом.

<картинка>

Если все $x$ и $y$ разные, то можно однозначно нарисовать в нем дерево: выберем в качестве корня самое высокое, разделим остальные точки на две части, построем деревья в них, объединим в одно большое.

**Высота дерева**. Высота в среднем логарифмическая. Любознательные могут ознакомиться с типа доказательством, но это не обязательно.

Глубина вершины — это количество родителей (прим. К. О.). Введем *индикатор*. $E[h] = \sum \frac{1}{k} \approx \log n $.

Если не объявлять указатели, в них будет записан 0. Внутри реализаций мы будем относиться к нулевому указателю как к пустому дереву.

In [ ]:
struct Node {
    int key, prior, size = 1;
    Node *l = 0, *r = 0;
    Node (int _key) { key = _key, prior = rand(); }
};

Для обработки операций мы поступим хитро: честно напишем всего две из них, а потом все будем выражать с помощью них.

**Merge**. Принимает два дерева, про которые известно, что в левом все вершины имеют меньший ключ, чем все в правом. Их нужно объединить так, чтобы ничего не сломалось.

In [ ]:
Node* merge (Node *l, Node *r) {
    if (!l) return r;
    if (!r) return l;
    if (l->prior > r->prior) {
        l->r = merge(l->r, r);
        upd(l);
        return l;
    }
    else {
        r->l = merge(l, r->l);
        upd(r);
        return r;
    }
}

**Split**. Принимает дерево и ключ, по которому его нужно разделить на два.

In [ ]:
typedef pair<Node*, Node*> Pair;

Pair split (Node *p, int x) {
    if (!p) return {0, 0};
    if (size(p->l) + 1 <= x) {
        Pair q = split(p->r, x-size(p->l)-1);
        p->r = q.first;
        upd(p);
        return {p, q.second};
    }
    else {
        Pair q = split(p->l, x);
        p->l = q.second;
        upd(p);
        return {q.first, p};
    }
}

Вот так будет выглядеть код, добавляющий $x$ в сет.

In [ ]:
Node *root = 0;

void insert (int x) {
    Pair q = split(root, x);
    Node *t = new Node(x);
    root = merge(q.first, merge(t, q.second));
}

# Неявный ключ

Очень часто требуется обрабатывать запросы, не привязанные к ключу. Например, «найди $k$-ый элемент».

Можно пересчитывать ключ на ходу. Для этого достаточно поддерживать размеры поддеревьев у каждой вершины.

Конечный код такой:

In [ ]:
struct Node {
    int key, prior, size = 1;
    Node *l = 0, *r = 0;
    Node (int _key) { key = _key, prior = rand(); }
};

int size (Node *v) { return v ? v->size : 0; }

void upd (Node *v) { v->size = 1 + size(v->l) + size(v->r); }

Node* merge (Node *l, Node *r) {
    if (!l) return r;
    if (!r) return l;
    if (l->prior > r->prior) {
        l->r = merge(l->r, r);
        upd(l);
        return l;
    }
    else {
        r->l = merge(l, r->l);
        upd(r);
        return r;
    }
}

typedef pair<Node*, Node*> Pair;

Pair split (Node *p, int x) {
    if (!p) return {0, 0};
    if (size(p->l) + 1 <= x) {
        Pair q = split(p->r, x-size(p->l)-1);
        p->r = q.first;
        upd(p);
        return {p, q.second};
    }
    else {
        Pair q = split(p->l, x);
        p->l = q.second;
        upd(p);
        return {q.first, p};
    }
}

Node *root = 0;

void insert (int x) {
    Pair q = split(root, x);
    Node *t = new Node(x);
    root = merge(q.first, merge(t, q.second));
}

# Персистентность

Персистентность можно реализовать так же, как и с деревом отрезков — просто каждый раз перед тем, как что-то сделать с вершиной, создать её копию и что-то делать уже с ней.

Применения у персистентности в ДД даже более интересные, чем в ДО. Например, можно строку представлять в виде ДД и делать быстрые операции копирования и вставки подстрок. Если прикрутить хэши, то можно делать ещё и сравнения.